# Retrogate Analysis

In [ ]:
%run ./nmm-game.ipynb
%run ./nmm-symmetry.ipynb

import pymongo
localMongo = pymongo.MongoClient("mongodb://localhost:27017/")
retroDb = localMongo["retro-database"]
retroCol = retroDb["retro-collection"]

In [ ]:
startBoard = ((' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '))

def generateBoards(board=startBoard, x=0, y=0):
    _boards  = []
    white = countStones(('', board), 'w')
    black = countStones(('', board), 'b')
    if (white == 3 and black == 3):
        return tuple([board])
    for i in range(x,3):
        for j in range(y,8):
            if board[i][j] == ' ':
                if white == 3:
                    newBoard = place(board, (i, j), 'b')
                    _boards += (generateBoards(newBoard, i, j))
                elif black == 3:
                    newBoard = place(board, (i, j), 'w')
                    _boards += generateBoards(newBoard, i, j)
                else:
                    for turn in ('w', 'b'):
                        newBoard = place(board, (i, j), turn)
                        _boards += generateBoards(newBoard, i, j)
    return _boards

In [ ]:
def getSymmetryUnique(boards):
    doubleBoards = set()
    uniqueStates = []
    stones = tuple([0, 0])
    for board in boards:
        state = tuple([stones, board])
        if not (state in doubleBoards):
            doubleBoards |= findSymmetries(('', board))
            uniqueStates.append(state)
    return uniqueStates

In [ ]:
def fillDb(states):
    fullStep = []
    stepCount = 0
    
    stepCount += 1
    _states = states
    for state in _states:
        _nextStates = nextStates(state, 'w')
        for ns in _nextStates:
            if (finished(ns, 'w') and utility(ns, 'w') == 1):
                entry = { "state": state, "nextState": ns, "steps": stepCount }
                retroCol.insert_one(entry)
                fullStep.append(state)
                states.remove(state)
                break
    
    while len(states) > 0:
        _states = states
        halfStep = []
        stepCount += 1
        for state in _states:
            _nextStates = nextStates(state, 'b')
            for ns in _nextStates:
                if (ns in fullStep):
                    halfStep.append(state)
                    states.remove(state)
                    break
        if halfStep == 0:
            break
        
        fullStep = []
        stepCount += 1
        for state in _states:
            _nextStates = nextStates(state, 'w')
            for ns in _nextStates:
                if (ns in halfStep):
                    entry = { "state": state, "nextState": ns, "steps": stepCount }
                    retroCol.insert_one(entry)
                    fullStep.append(state)
                    states.remove(state)
                    break
        if fullStep == 0:
            break

In [ ]:
boards = generateBoards()
states = getSymmetryUnique(boards)
fillDb(states)